# Data prep


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [4]:
df = pd.read_excel('../AWI_data_analysis_assessment.xlsm', sheet_name = 'AWI_data_RAW')

In [5]:
# Defining functions

def check_if_present(FIELD):
    if FIELD == 1 or FIELD ==2 or FIELD ==6:
        return 1
    else:
        return 0
    
def check_if_zero(FIELD):
    if FIELD == 0:
        return 1
    else:
        return 0
    
def check_if_name(FIELD):
    if FIELD.strip() == '':
        return 0
    else:
        return 1


In [6]:
# Columns to check if if parcel has been marked as woodland
columns_to_assess = ['TITHE','C19_EM', 'OSD', 'CM_1773', 'C18_EM', 'C17_EM', 'EM','OTHER', 'HE_REF' ]

#Loop through each parcel and check if woodland present in source, and if there are any zeros (not woodland)
for i in columns_to_assess:
    df[f"present_{i}"] = df[i].apply(check_if_present)
    df[f"zero_{i}"] = df[i].apply(check_if_zero)

In [7]:
# Combined present to see how many sources show woodland present
df['present_combined'] = df['present_TITHE']+df['present_C19_EM'] +df['present_OSD']+df['present_CM_1773'] +df['present_C18_EM']+df['present_C17_EM'] +df['present_EM']+df['present_OTHER'] +df['present_HE_REF']
df['present_primary'] = df['present_TITHE'] + df['present_OSD'] + df['present_C17_EM']+df['present_HE_REF']
df['present_secondary'] =df['present_C19_EM']+df['present_CM_1773'] +df['present_C18_EM'] +df['present_EM']+df['present_OTHER'] 

# Combine zero to show how many sources show no woodland present
df['zero_combined'] = df['zero_TITHE']+df['zero_C19_EM'] +df['zero_OSD']+df['zero_CM_1773'] +df['zero_C18_EM']+df['zero_C17_EM'] +df['zero_EM']+df['zero_OTHER'] +df['zero_HE_REF']
df['zero_primary'] = df['zero_TITHE']+df['zero_OSD']
df['zero_secondary'] =  df['zero_C19_EM'] +df['zero_CM_1773'] +df['zero_C18_EM']+df['zero_C17_EM'] +df['zero_EM']+df['zero_OTHER'] +df['zero_HE_REF']


In [8]:
# Check which parcels are named

df['NAME_TITHE'] = df['NAME_TITHE'].astype(str)

names_to_assess = ['NAME_TITHE','NAME_EP1', 'NAME_OSD', 'NAME_EM']

for i in names_to_assess:
    df[f"present_{i}"] = df[i].apply(check_if_name)

# Combine to get total named sources for parcel    
df['name_present'] = df['present_NAME_EM']+df['present_NAME_OSD']+df['present_NAME_EP1'] +df['present_NAME_TITHE']

# Decision tree


In [ ]:

df_primary = df[df.present_primary >=1]
## A - Ancient woodland
## Contains primary evidence and no zero's present
df_primary_no_zero = df_primary[df_primary.zero_combined ==0]
df_primary_no_zero['Category'] = 'Ancient woodland'
df_primary_no_zero['subCategory'] = 'A'

df_primary_one_zero = df_primary[df_primary.zero_combined ==1]
## A - Not ancient woodland 
## Has primary evidence but contains more than one zero
df_primary_plural_zero = df_primary[df_primary.zero_combined >1]
df_primary_plural_zero['Category'] = 'Not ancient woodland'
df_primary_plural_zero['subCategory'] = 'A'
## B - Ancient woodland
## Has primary evidence, one zero present, but is a named woodland 
df_primary_one_zero_named = df_primary_one_zero[df_primary_one_zero.name_present >=1]
df_primary_one_zero_named['Category'] = 'Ancient woodland'
df_primary_one_zero_named['subCategory'] = 'B'
## A- Maybe ancient woodland
## Has primary evidence, one zero present and is not a named woodland. 
### Further research needed, what source is zero?
df_primary_one_zero_not_named = df_primary_one_zero[df_primary_one_zero.name_present==0]
df_primary_one_zero_not_named['Category'] = 'Maybe ancient woodland'
df_primary_one_zero_not_named['subCategory'] = 'A'


In [117]:
df_primary.shape

(5043, 86)

In [ ]:
df_secondary = df[(df.present_primary ==0)&(df.present_secondary >=1)]
df_no_evidence = df[(df.present_primary ==0)&(df.present_secondary ==0)]

## D - Maybe ancient woodland
## No evidence, but contains no zeros, so may just not have been covered by maps
# Contains lots of 3/4's (no map coverage), requires further investigation to see if any maps preset
df_no_evidence_no_zero = df_no_evidence[df_no_evidence.zero_combined ==0]
df_no_evidence_no_zero['Category'] = 'Maybe ancient woodland'
df_no_evidence_no_zero['subCategory'] = 'D'

## C - Not ancient woodland
## No evidence, and contains at least one zero, so assumed not woodland. 
## 1637, ~600 1 zero, ~1000 more than one zero
df_no_evidence_zero = df_no_evidence[df_no_evidence.zero_combined >= 1]
df_no_evidence_zero['Category'] = 'Not ancient woodland'
df_no_evidence_zero['subCategory'] = 'C'

df_no_evidence_one_zero = df_no_evidence[df_no_evidence.zero_combined == 1]
df_no_evidence_plural_zero = df_no_evidence[df_no_evidence.zero_combined > 1]



In [ ]:
df_secondary_no_zero = df_secondary[df_secondary.zero_combined ==0]
## C - Ancient woodland 
## No primary evidence, but contains secondary evidence, no zero's present and is named. 
df_secondary_no_zero_named = df_secondary_no_zero[df_secondary_no_zero.name_present >=1]
df_secondary_no_zero_named['Category'] = 'Ancient woodland'
df_secondary_no_zero_named['subCategory'] = 'C'
## B- Maybe ancient woodland
## No primary evidence, but contains secondary evidence, no zero's present, but is not named
## Further research needed; why is not present on primary evidence? 
df_secondary_no_zero_not_named  = df_secondary_no_zero[df_secondary_no_zero.name_present ==0]
df_secondary_no_zero_not_named['Category'] = 'Maybe ancient woodland'
df_secondary_no_zero_not_named['subCategory'] = 'B'

df_secondary_one_zero = df_secondary[df_secondary.zero_combined ==1]
## C - Maybe ancient woodland
## No primary evidence, secondary evidence, but one zero present, and named
## Further research; why no primary evidence? and what source is zero? What source is name? 
df_secondary_one_zero_named  = df_secondary_one_zero[df_secondary_one_zero.name_present >=1]
df_secondary_one_zero_named['Category'] = 'Maybe ancient woodland'
df_secondary_one_zero_named['subCategory'] = 'C'
## E - Maybe ancient woodland
## No primary evidence, secondary evidence, but one zero present, not named
## Probably not ancient woodland, but check primary evidence, and zero. 
df_secondary_one_zero_not_named  = df_secondary_one_zero[df_secondary_one_zero.name_present ==0]
df_secondary_one_zero_not_named['Category'] = 'Maybe ancient woodland'
df_secondary_one_zero_not_named['subCategory'] = 'E'
## B - Not ancient woodland
df_secondary_plural_zero = df_secondary[df_secondary.zero_combined >1]
df_secondary_plural_zero['Category'] = 'Not ancient woodland'
df_secondary_plural_zero['subCategory'] = 'B'



In [103]:
df_secondary_no_zero_not_named.shape

(34, 88)

In [25]:


df_combined = pd.concat([df_primary_no_zero,
df_primary_plural_zero,
df_primary_one_zero_named,
df_primary_one_zero_not_named,
df_no_evidence_no_zero,
df_no_evidence_zero,
df_secondary_no_zero_named,
df_secondary_no_zero_not_named,
df_secondary_one_zero_named,
df_secondary_one_zero_not_named,
df_secondary_plural_zero,])

In [31]:
df_ancient_only = df_combined[df_combined.Category == 'Ancient woodland']
df_maybe_ancient = df_combined[df_combined.Category == 'Maybe ancient woodland']
df_not_ancient = df_combined[df_combined.Category == 'Not ancient woodland']

In [51]:
df_not_ancient_grouped = df_not_ancient.groupby('P3_Status').count().reset_index()
df_ancient_grouped = df_ancient_only.groupby('P3_Status').count().reset_index()

In [61]:
df_not_ancient_grouped = df_not_ancient.groupby('subCategory').count().reset_index()


In [62]:
df_not_ancient_grouped

,subCategory,NAME_OS,NAME_EP1,P3_UID,P3_IGNORE,C21_AP,EPOCH_2,EPOCH_1,C20_AP,C20_AP_REF,...,present_secondary,zero_combined,zero_primary,zero_secondary,present_NAME_TITHE,present_NAME_EP1,present_NAME_OSD,present_NAME_EM,name_present,Category
0,A,914,914,914,914,914,914,914,914,914,...,914,914,914,914,914,914,914,914,914,914
1,B,24,24,24,24,24,24,24,24,24,...,24,24,24,24,24,24,24,24,24,24
2,C,1637,1637,1637,1637,1637,1637,1637,1637,1637,...,1637,1637,1637,1637,1637,1637,1637,1637,1637,1637


In [69]:
fig = px.bar(df_ancient_grouped, x ='P3_Status', y = 'NAME_OS' )
fig.show()

In [59]:
fig = px.bar(df_ancient_grouped, x ='P3_Status', y = 'NAME_OS' )
fig.show()

In [147]:
df_sankey = pd.read_csv('../sankey lookup.csv')

In [148]:


fig = go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label =df_sankey['name2'],
      color = df_sankey['node_colour']
    ),
    link = dict(
      source = df_sankey['source'], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = df_sankey['target'],
      value = df_sankey['value'],
      color = df_sankey['colour']
  ))])

fig.update_layout(title_text="Basic Sankey Diagram", font_size=10)
fig.show()